# Studying the ImageNet100 dataset
Our goal in this study is to utilize the ResNet-50 neural networks architecture, pretrained on the ImageNet dataset, in order to test the accuracy on validation sets. After that, the goal of the study will be to improve the model's prowess by altering images using data augmentation and pertubation within the latent space.

## Setting up
Having downloaded the validation set for ImageNet100, a subset of the ImageNet dataset with 100 labels, we can begin setting up our environment to test the control performance of the model.

In [1]:
from google.colab import drive
import sys

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision.models import resnet50, ResNet50_Weights
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import cv2
from PIL import Image
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import re
import kagglehub
import shutil
import os, json
import matplotlib.pyplot as plt
import seaborn as sns

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Organizing image data
We are going to be structuring our image paths in a DataFrame within the notebook environment in order to ensure fast access to the images and their labels. Additionally, it allows us to look at paths and labels to find possible discrepancies in the predictions provided and the labels we currently have.

In [3]:
dataset_path = '/content/drive/MyDrive/CS682/Data Augmentation Project/ImageNet/8/'
labels_path = dataset_path + 'Labels.json'

labels = []
with open(labels_path, 'r') as f:
    data = json.load(f)
    labels = data

images = []
for label in os.listdir(f'{dataset_path}val.X'):
    actual_label = labels[label]
    ind = actual_label.find(',')
    if ind != -1:
        actual_label = actual_label[:ind]
    for filename in os.listdir(f'{dataset_path}val.X/{label}'):
        images.append({
            'path': f'{dataset_path}val.X/{label}/{filename}',
            'label': actual_label
        })

df = pd.DataFrame(images)

# Sanity check on paths
print(df['path'][:5],'\n', df['label'][:5])

0    /content/drive/MyDrive/CS682/Data Augmentation...
1    /content/drive/MyDrive/CS682/Data Augmentation...
2    /content/drive/MyDrive/CS682/Data Augmentation...
3    /content/drive/MyDrive/CS682/Data Augmentation...
4    /content/drive/MyDrive/CS682/Data Augmentation...
Name: path, dtype: object 
 0    great white shark
1    great white shark
2    great white shark
3    great white shark
4    great white shark
Name: label, dtype: object


## Downloading ResNet
We can now begin downloading the ResNet50 model, along with its pre-trained weights we can use to preprocess our image data to ensure data shape compatibility with our input and model architecture.

In [5]:
res_weights = ResNet50_Weights.DEFAULT
model = resnet50(weights = res_weights)
model.fc = nn.Linear(model.fc.in_features, 100)
model = model.to(device)
model.eval()

preprocess = res_weights.transforms()
cats = res_weights.meta["categories"]



Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s]


In [ ]:
# Sanity check on categories
for i in range(0, len(cats), 10):
    print(', '.join(map(str, cats[i:i+10])))
    print()

tench, goldfish, great white shark, tiger shark, hammerhead, electric ray, stingray, cock, hen, ostrich

brambling, goldfinch, house finch, junco, indigo bunting, robin, bulbul, jay, magpie, chickadee

water ouzel, kite, bald eagle, vulture, great grey owl, European fire salamander, common newt, eft, spotted salamander, axolotl

bullfrog, tree frog, tailed frog, loggerhead, leatherback turtle, mud turtle, terrapin, box turtle, banded gecko, common iguana

American chameleon, whiptail, agama, frilled lizard, alligator lizard, Gila monster, green lizard, African chameleon, Komodo dragon, African crocodile

American alligator, triceratops, thunder snake, ringneck snake, hognose snake, green snake, king snake, garter snake, water snake, vine snake

night snake, boa constrictor, rock python, Indian cobra, green mamba, sea snake, horned viper, diamondback, sidewinder, trilobite

harvestman, scorpion, black and gold garden spider, barn spider, garden spider, black widow, tarantula, wolf spide

## Validation set Predictions
We will now run the ResNet50 architecture on the ImageNet100 dataset. Notice, that the pre-trained model is trained on the ImageNet1k set, which is the original, much larger, dataset that ImageNet100 is derived from.

In [6]:
def load_image(image_path: str):
    img = preprocess(Image.open(image_path))
    img = img.unsqueeze(0)
    return img

indices_predictions = {}
label_trav = []

# Beginning the baseline predictions check
predictions = []
for i, row in df.iterrows():
    p, l = row['path'], row['label']
    # print(l)
    label_trav.append(l)
    batch = load_image(p)
    # Use and predict through the model
    with torch.no_grad():
        outputs = model(batch)
        _, predicted = torch.max(outputs, 1)
    pred_class = cats[predicted]
    indices_predictions[predicted.item()] = pred_class
    predictions.append(pred_class)

# print(cats)
# Adding the predictions to the data frame
for key in sorted(list(indices_predictions.keys())):
    print(key, indices_predictions[key])
print(len(indices_predictions))

# df['predictions'] = predictions

KeyboardInterrupt: 

Now that the model has run its course, we can run our metrics on the performance of the pretrained model to determine its performance. We can do this through the usage of ```scikit learn``` metric functions such accuracy and f_1 scores.

In [ ]:
print(accuracy_score(df['label'], df['predictions']))
print(precision_score(df['label'], df['predictions'], zero_division = 0.0,
                      average = 'weighted'))
print(recall_score(df['label'], df['predictions'], zero_division = 0.0,
                   average = 'weighted'))
print(f1_score(df['label'], df['predictions'], zero_division = 0.0,
               average = 'weighted'))

## Generating a Classification Report and Displaying data
We see that our metrics above, but what if we want to determine precision and recall for each individual class and study how the metrics vary for all the classes side by side?

We will be using the classification report generate by ```sklearn```, and graphs generated by ```matplotlib``` and ```seaborn```.

In [ ]:
report = classification_report(df['label'], df['predictions'], output_dict = True, zero_division = 0.0)
report = pd.DataFrame(report).transpose()
report = report[report['support'] != 0]

print(report)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8, 5))

g = sns.barplot(x = report.index, y = report['precision'])
g.set_ylabel("PRECISION")
g.set_xlabel('CLASSES')
g.set_xticks([])

plt.show()